## Setup

To use SMTLink in an ACL2 book, you need:

```lisp
;; Include the SMTLink book
(include-book "projects/smtlink/top" :dir :system)

;; Enable tshell (required for calling external Z3)
(value-triple (tshell-ensure))

;; Add the computed hint
(add-default-hints '((SMT::SMT-computed-hint clause)))
```

The `smtlink-config` file in `$SMT_HOME` or `$HOME` specifies the Python command:
```
smt-cmd=/usr/bin/env python
```

In [3]:
;; Include the SMTLink book
(include-book "projects/smtlink/top" :dir :system :ttags :all)


TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

TTAG NOTE (for included book): Adding ttag :QUICKLISP.SHELLPOOL from book /home/acl2/books/quicklisp/shellpool.

TTAG NOTE (for included book): Adding ttag :TSHELL from book /home/acl2/books/projects/smtlink/trusted/run.

TTAG NOTE (for included book): Adding ttag :TSHELL from book /home/acl2/books/projects/smtlink/trusted/write.

TTAG NOTE (for included book): Adding ttag :TSHELL from book /home/acl2/books/projects/smtlink/trusted/prove.

TTAG NOTE (for included book): Adding ttag :TSHELL from book /home/acl2/books/centaur/misc/tshell.

TTAG NOTE (for included book): Adding ttag :READ-STRING from book /home/acl2/books/projects/smtlink/trusted/run.

TTAG NOTE (for included book): Adding ttag :READ-STRING from book /home/acl2/books/std/io/read-string.

TTAG NOTE (for included book): Adding ttag :WRITES-OKP from book /home/acl2/books/projects/smtlink/trusted/write.

TTAG NOTE (for included 

In [4]:
;; Enable tshell (required for calling external Z3)
(value-triple (tshell-ensure))

 NIL


In [5]:
;; Add the computed hint
(add-default-hints '((SMT::SMT-computed-hint clause)))

 ((SMT::SMT-COMPUTED-HINT CLAUSE)
  (SET::PICK-A-POINT-SUBSET-HINT ID CLAUSE
                                 WORLD STABLE-UNDER-SIMPLIFICATIONP))


## Example 1: Basic Polynomial Inequality

**Theorem:** For all $x, y \in \mathbb{R}$, if $\frac{9x^2}{8} + y^2 \le 1$ and $x^2 - y^2 \le 1$, then $y < 3(x - \frac{17}{8})^2 - 3$.

This is a non-linear polynomial inequality that ACL2's rewriter cannot handle directly, but Z3 can prove it easily.

```lisp
;; Helper function
(defun x^2-y^2 (x y) (- (* x x) (* y y)))

;; The theorem
(defthm poly-ineq-example
  (implies (and (real/rationalp x) (real/rationalp y)
                (<= (+ (* (/ 9 8) x x) (* y y)) 1)
                (<= (x^2-y^2 x y) 1))
           (< y (- (* 3 (- x (/ 17 8)) (- x (/ 17 8))) 3)))
  :hints(("Goal" :smtlink nil)))
```

### Key Points

1. `:smtlink nil` - No special hints needed for basic cases
2. Z3 handles the polynomial arithmetic automatically
3. The `x^2-y^2` function is expanded by SMTLink before sending to Z3

In [6]:
;; Helper function
(defun x^2-y^2 (x y) (- (* x x) (* y y)))

;; The theorem
(defthm poly-ineq-example
  (implies (and (real/rationalp x) (real/rationalp y)
                (<= (+ (* (/ 9 8) x x) (* y y)) 1)
                (<= (x^2-y^2 x y) 1))
           (< y (- (* 3 (- x (/ 17 8)) (- x (/ 17 8))) 3)))
  :hints(("Goal" :smtlink nil)))


Since X^2-Y^2 is non-recursive, its admission is trivial.  We observe
that the type of X^2-Y^2 is described by the theorem 
(ACL2-NUMBERP (X^2-Y^2 X Y)).  We used primitive type reasoning.

Summary
Form:  ( DEFUN X^2-Y^2 ...)
Rules: ((:FAKE-RUNE-FOR-TYPE-SET NIL))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 X^2-Y^2


Using clause-processor Smtlink
Goal'
Goal''
Goal'''
Goal'4'
Subgoal 2
Subgoal 2'
Subgoal 2.2
Subgoal 2.2'
Subgoal 2.2''
Subgoal 2.2'''
Using default SMT-trusted-cp...
; SMT solver: `python /tmp/py_file/smtlink.V64In`: 0.06 sec, 0 bytes
Proved!
Subgoal 2.2'4'
Subgoal 2.2'5'
Subgoal 2.1
Subgoal 2.1'
Subgoal 2.1''
Subgoal 2.1'''
Subgoal 1
Subgoal 1'
Subgoal 1''
Subgoal 1'''

Q.E.D.

Summary
Form:  ( DEFTHM POLY-INEQ-EXAMPLE ...)
Rules: ((:DEFINITION HIDE)
        (:DEFINITION NOT)
        (:DEFINITION SYNP)
        (:DEFINITION SMT::TYPE-HYP)
        (:DEFINITION X^2-Y^2)
        (:EXECUTABLE-COUNTERPART BINARY-*)
        (:EXECUTABLE-COUNTERPART BINARY-+)
 

## Understanding SMTLink Output

When SMTLink proves a theorem, you'll see output like:

```
Using clause-processor Smtlink
Goal'
Goal''
SMT-goal-generator=>Expanding ... X^2-Y^2
Subgoal 2
Subgoal 2.2
Subgoal 2.2'
Using default SMT-trusted-cp...
; SMT solver: `python /tmp/py_file/smtlink.w59zR`: 0.52 sec, 7,904 bytes
Proved!
```

### What's Happening:

1. **Function expansion**: `SMT-goal-generator=>Expanding ... X^2-Y^2`
2. **Subgoals**: ACL2 generates auxiliary clauses for soundness
3. **SMT solving**: The trusted clause processor calls Z3
4. **Result**: "Proved!" means Z3 verified the goal

## Example 2: Counter-Example Detection

When Z3 finds a counter-example, SMTLink reports it:

```lisp
;; This is NOT a theorem!
(defthm non-theorem
  (implies (and (real/rationalp x)
                (real/rationalp y)
                (integerp (/ x y)))
           (not (equal y 0)))
  :hints(("Goal" :smtlink nil))
  :rule-classes nil)
```

This fails because in ACL2, `(/ x 0) = 0`, so `(integerp (/ x 0))` is true!

SMTLink fails with:
```
HARD ACL2 ERROR in SMT-TRANSLATOR=>TRANSLATE-FUNCTION: 
Not a basic SMT function: INTEGERP
```

This is because `integerp` used as a constraint (not type declaration) isn't directly supported.

In [7]:
(defthm non-theorem
  (implies (and (real/rationalp x)
                (real/rationalp y)
                (integerp (/ x y)))
           (not (equal y 0)))
  :hints(("Goal" :smtlink nil))
  :rule-classes nil)



Using clause-processor Smtlink
Goal'
Goal''
Goal'''
Goal'4'
Subgoal 2
Subgoal 2'
Subgoal 2.2
Subgoal 2.2'
Subgoal 2.2''
Subgoal 2.2'''
Using default SMT-trusted-cp...


HARD ACL2 ERROR in SMT::SMT-TRANSLATOR=>TRANSLATE-FUNCTION:  Not a
basic SMT function: INTEGERP




ACL2 Error [Evaluation] in ( DEFTHM NON-THEOREM ...):  Evaluation aborted.
To debug see :DOC print-gv, see :DOC trace, and see :DOC wet.



ACL2 Error [Prove] in ( DEFTHM NON-THEOREM ...):  Evaluation failed
for the :clause-processor hint.


Summary
Form:  ( DEFTHM NON-THEOREM ...)
Rules: NIL
Hint-events: ((:CLAUSE-PROCESSOR SMT::ADD-HYPO-CP)
              (:CLAUSE-PROCESSOR SMT::EXPAND-CP-FN)
              (:CLAUSE-PROCESSOR SMT::PROCESS-HINT)
              (:CLAUSE-PROCESSOR SMT::REMOVE-HINT-PLEASE)
              (:CLAUSE-PROCESSOR SMT::TYPE-EXTRACT-FN)
              (:CLAUSE-PROCESSOR SMT::UNINTERPRETED-FN-CP))
Time:  0.02 seconds (prove: 0.02, print: 0.00, other: 0.00)

*** Note: No checkpoints to print. ***

ACL2 E

## SMTLink Hint Keywords

The `:smtlink` hint accepts several keywords:

| Keyword | Purpose |
|---------|----------|
| `:functions` | Declare uninterpreted functions with types |
| `:hypotheses` | Add extra hypotheses (must be proven as subgoals) |
| `:int-to-rat` | Treat integers as rationals (helps Z3) |
| `:fty` | Specify FTY types used in the theorem |
| `:abstract` | Declare abstract predicates |

### Basic Usage

```lisp
;; Most basic - let SMTLink handle everything
:hints (("Goal" :smtlink nil))

;; With custom hints
:hints (("Goal" :smtlink (:int-to-rat t)))
```

## Example 3: Functions as Uninterpreted

Sometimes you want Z3 to treat a function abstractly:

```lisp
(define foo ((x real/rationalp))
  :returns (rx real/rationalp)
  (b* ((x (realfix x)))
    (+ (* x x) 1)))

(defthm poly-ineq-example-functions
  (implies (and (real/rationalp x))
           (< 0 (* 2 (foo x))))
  :hints(("Goal"
          :smtlink
          (:functions ((foo :formals ((x real/rationalp))
                            :returns ((rx real/rationalp))
                            :level 0))
           :hypotheses (((<= 1 (foo x))
                         :hints (:in-theory (enable foo))))))))
```

### Key Points:

- `:level 0` makes `foo` uninterpreted (Z3 knows nothing about its definition)
- `:hypotheses` adds `(<= 1 (foo x))` which must be proven separately
- The hypothesis has its own `:hints` for ACL2 to prove it

In [8]:
(define foo ((x real/rationalp))
  :returns (rx real/rationalp)
  (b* ((x (realfix x)))
    (+ (* x x) 1)))

(defthm poly-ineq-example-functions
  (implies (and (real/rationalp x))
           (< 0 (* 2 (foo x))))
  :hints(("Goal"
          :smtlink
          (:functions ((foo :formals ((x real/rationalp))
                            :returns ((rx real/rationalp))
                            :level 0))
           :hypotheses (((<= 1 (foo x))
                         :hints (:in-theory (enable foo))))))))


Since FOO is non-recursive, its admission is trivial.  We observe that
the type of FOO is described by the theorem 
(AND (RATIONALP (FOO X)) (< 0 (FOO X))).  We used primitive type reasoning
and the :type-prescription rules NONNEGATIVE-PRODUCT and REALFIX.

Computing the guard conjecture for FOO....

The guard conjecture for FOO is trivial to prove, given the :type-
prescription rules NONNEGATIVE-PRODUCT and REALFIX.  FOO is compliant
with Common Lisp.

Summary
Form:  ( DEFUN FOO ...)
Rules: ((:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:TYPE-PRESCRIPTION NONNEGATIVE-PRODUCT)
        (:TYPE-PRESCRIPTION REALFIX))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM REAL/RATIONALP-OF-FOO
          (B* ((RX (FOO X))) (REAL/RATIONALP RX))
          :HINTS ((STD::RETURNSPEC-DEFAULT-DEFAULT-HINT 'FOO
                                                        ID WORLD))
          :RULE-CLASSES :REWRITE)

ACL2 Warning [Non-rec] in ( DEFTHM REAL/RATIONALP-OF-FOO ...):  A :REWR

## Next Steps

See the following notebooks for more advanced topics:

1. **02-smtlink-expt.ipynb** - Custom Z3 modules for `expt`
2. **03-smtlink-fty.ipynb** - Using FTY types with SMTLink
3. **04-smtlink-hardware.ipynb** - Ring oscillator verification example

## Summary

SMTLink extends ACL2's reasoning power by:

1. **Non-linear arithmetic**: Polynomial inequalities
2. **Bit-vector reasoning**: Hardware verification
3. **Abstract types**: Via FTY integration
4. **Counter-examples**: Debugging false conjectures

The trust model relies on Z3 being sound - ACL2 trusts the trusted clause processor.